In [3]:
# Import relevant Python libraries here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load dataset into notebook
df = pd.read_csv('Marriage Perception (Responses) - Form Responses 1.csv')

# Shape of dataset
print(f"Size of Dataset (Rows, Cols): {df.shape}\n")

# Peek at head of dataset
print("Preview: ")
df.head()

Size of Dataset (Rows, Cols): (92, 29)

Timestamp                                                                                                                 str
Please rate each of the following: [I want to get married.]                                                               str
Please rate each of the following: [Marriage comes with unique benefits.]                                                 str
Please rate each of the following: [Everyone should get married.]                                                         str
Please rate each of the following: [Marriage is more than a legal document.]                                              str
Please rate each of the following: [I believe marriage is an outdated idea.]                                              str
Please rate each of the following: [People can be happy and not get married.]                                             str
Please rate each of the following: [I think relationships should have the goal

,Timestamp,Please rate each of the following: [I want to get married.],Please rate each of the following: [Marriage comes with unique benefits.],Please rate each of the following: [Everyone should get married.],Please rate each of the following: [Marriage is more than a legal document.],Please rate each of the following: [I believe marriage is an outdated idea.],Please rate each of the following: [People can be happy and not get married.],Please rate each of the following: [I think relationships should have the goal of getting married.],Please rate each of the following: [Marriage comes with unique challenges. ],Please rate each of the following: [I feel marriage is the highest level in a relationship.],...,Please rate each of the following: [I think variety is the spice of life.],Please rate each of the following: [I try to have my life and career clearly mapped out.],Please rate each of the following: [Taking chances is part of life.],How old are you?,What is your gender?,What is your sexual orientation?,Are you married?,"Are you religious? If so, what is your affiliation?",How religious are you?,"If you have any questions or comments, leave it down below."
0,2/19/2026 9:04:51,Strongly Agree (7),Strongly Agree (7),2,Strongly Agree (7),Strongly Disagree (1),5,6,Strongly Agree (7),Strongly Agree (7),...,6,6,6,59,Female,Heterosexual / Straight,Yes,Christian,6,NaN
1,2/19/2026 9:05:31,5,6,2,6,4,6,3,6,3,...,2,5,4,21,Male,Heterosexual / Straight,No,Christianity,2,NaN
2,2/19/2026 10:01:42,3,3,3,3,Strongly Disagree (1),3,3,3,2,...,2,Strongly Disagree (1),2,51,Female,Heterosexual / Straight,Yes,Protestant,7,NaN
3,2/19/2026 10:04:15,5,4,Strongly Disagree (1),5,5,Strongly Agree (7),5,Strongly Agree (7),5,...,6,2,6,20,Female,Heterosexual / Straight,No,No,3,NaN
4,2/19/2026 10:04:51,Strongly Agree (7),4,Strongly Disagree (1),Strongly Agree (7),3,Strongly Agree (7),Strongly Disagree (1),Strongly Agree (7),5,...,6,4,6,21,Female,Heterosexual / Straight,No,Episcopalian,4,NaN


In [ ]:
# Check for duplicates
print(f"Duplicates:  {df.duplicated().sum()}\n")

# Check for types and missing entries
df.info()

Duplicates:  0

<class 'pandas.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 29 columns):
 #   Column                                                                                                                Non-Null Count  Dtype
---  ------                                                                                                                --------------  -----
 0   Timestamp                                                                                                             92 non-null     str  
 1   Please rate each of the following: [I want to get married.]                                                           92 non-null     str  
 2   Please rate each of the following: [Marriage comes with unique benefits.]                                             92 non-null     str  
 3   Please rate each of the following: [Everyone should get married.]                                                     92 non-null     str  
 4   Please rate each 